In [0]:
storageAccountKey = dbutils.secrets.get(scope = 'databricksScope', key = 'storage-account-access-key')

alreadyMounted = False
for x in dbutils.fs.mounts():
	if x.mountPoint == "/mnt/sales":
		alreadyMounted = True
		break

if not alreadyMounted:
		dbutils.fs.mount(source = 'wasbs://sales@salesprojectsa123.blob.core.windows.net',
                 mount_point = '/mnt/sales',
                 extra_configs = {'fs.azure.account.key.salesprojectsa123.blob.core.windows.net':storageAccountKey})
		print("mount successful")
else:
	print("Already mounted")




mount successful


In [0]:

filename = dbutils.widgets.get('filename')


orders  = spark.read.csv("/mnt/sales/landing/{}".format(filename), inferSchema=True, header=True)


#check if order_id column is distinct or not
errorFlag = False
ordersCount = orders.count()
orderIdCount = orders.select("order_id").distinct().count()

if ordersCount != orderIdCount:
    errorFlag = True

if errorFlag:
    dbutils.fs.mv("/mnt/sales/landing/{}".format(filename),'/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlag":"true", "errorMessage":"Order ID is repeated"}')

orders.createOrReplaceTempView("orders_table")


#connecting to sql database
dbServer= 'sales-project-sql-server'
dbPort = '1433'
dbName= 'sales-project'
dbUser= 'ramireddy'
databricksScope= 'sales-project-scope'

connectionUrl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer,dbPort, dbName, dbUser)
dbPassword = dbutils.secrets.get(scope = databricksScope, key= 'sql-password')

connectionProperties = {
'password': dbPassword,
'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}


valid_order_status = spark.read.jdbc(url=connectionUrl, table='dbo.valid_order_status', properties=connectionProperties)

valid_order_status.createOrReplaceTempView('valid_status')

invalidIdRows = spark.sql("select * from orders_table where order_status not in (select * from valid_status)")


if invalidIdRows.count() > 0:
    errorFlag = True 

if errorFlag:
    dbutils.fs.mv("/mnt/sales/landing/{}".format(filename),'/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlag":"true", "errorMessage":"Invalid Order Status"}')
else:
    dbutils.fs.mv("/mnt/sales/landing/{}".format(filename),'/mnt/sales/staging')
    dbutils.notebook.exit('{"errorFlag":"false", "errorMessage":"No Errors"}')


